## setup

Note: Before running this notebook, populate the database:

```
$ python repopulate_db.py --email <your_email>
```

All API endpoints that modify the database (rather than just fetching data from it) take a `test` argument, and if `test` is `False`, no changes to the database are made. All calls here will use the test parameter.

In [1]:
import io
import os
from pprint import pprint

import requests

In [2]:
BASE_URL = 'http://localhost:5000/'

In [3]:
def call_endpoint(method, endpoint,
                  auth=None, verbose=True, return_json=True, **kwargs):
    """
    Args:
        method ({'GET', 'PUT', 'POST', 'DELETE'})
        endpoint (str)
        auth (Tuple[str, str])
        verbose (bool)
        return_json (bool)
        **kwargs
    
    Returns:
        ``requests.response`` or JSON
    """
    url = BASE_URL + endpoint
    response = requests.request(method, url, auth=auth, **kwargs)

    if verbose is True:
        print(method, '=>', response.url)
    if return_json is True:
        return response.json()
    else:
        return response


def get_auth_token(email, password):
    return (call_endpoint('authtoken', 'get', (email, password))['token'], '')

In [16]:
auth = (call_endpoint('GET', 'authtoken', auth=('burtdewilde@gmail.com', 'password'))['token'], '')

GET => http://localhost:5000/authtoken


---

## USERS

### register a new user

input: user name, email and password

output: user info with `id` and `created_at` fields filled in by the database

In [38]:
user_info = {'name': 'Burton DeWilde',
             'email': 'burtdewilde@gmail.com',
             'password': 'password'}

call_endpoint('POST', 'users',
              json=user_info, params={'test': True})

POST => http://localhost:5000/users?test=True


{'created_at': None,
 'email': 'burtdewilde@gmail.com',
 'id': None,
 'name': 'Burton DeWilde'}

### authenticate an existing user

input: user email and password

output: authentication token, expires in 30 minutes

In [18]:
login_creds = ('burtdewilde@gmail.com', 'password')

response = call_endpoint('GET', 'authtoken', auth=login_creds)
auth = (response['token'], '')
auth

GET => http://localhost:5000/authtoken


('eyJhbGciOiJIUzI1NiIsImlhdCI6MTQ3NTIwMDY0MCwiZXhwIjoxNDc1MjAyNDQwfQ.eyJpZCI6MX0.5a9cvcopK32Tcgu5UqVF3UFD4SbK87oQ2-gALfRvZb4',
 '')

### get user's info by id

input: user id

output: user info, or a subset specified by the `fields` parameter (**note:** `id` is always returned)

In [19]:
call_endpoint('GET', 'users/{id}'.format(id=1), auth=auth)

GET => http://localhost:5000/users/1


{'created_at': '2016-09-29T23:52:27.781970+00:00',
 'email': 'burtdewilde@gmail.com',
 'id': 1,
 'name': 'Burton DeWilde'}

In [20]:
call_endpoint('GET', 'users/{id}'.format(id=1), auth=auth,
              params={'fields': 'email,name'})

GET => http://localhost:5000/users/1?fields=email%2Cname


{'email': 'burtdewilde@gmail.com', 'id': 1, 'name': 'Burton DeWilde'}

### get user's info by email

if you don't know the user's id

intput: user email

output: user info, or a subset

In [21]:
call_endpoint('GET', 'users', auth=auth,
              params={'email': 'burtdewilde@gmail.com'})

GET => http://localhost:5000/users?email=burtdewilde%40gmail.com


{'created_at': '2016-09-29T23:52:27.781970+00:00',
 'email': 'burtdewilde@gmail.com',
 'id': 1,
 'name': 'Burton DeWilde'}

### delete a user

input: user id

output: nothing

**note:** this also deletes all reviews owned by user, so be very careful

In [22]:
call_endpoint('DELETE', 'users/{id}'.format(id=1), auth=auth,
              params={'test': True})

DELETE => http://localhost:5000/users/1?test=True


---

## REVIEWS

### create a new review

input: review name and description

output: review info, plus `id`, `created_at`, `status`, `num_citation_screening_reviewers`, and `num_fulltext_screening_reviewers` fields filled in by the database

In [23]:
review_info = {'name': "Burton's Great Review",
               'description': 'The greatest systematic review ever created.'}

call_endpoint('POST', 'reviews', auth=auth,
              json=review_info, params={'test': True})

POST => http://localhost:5000/reviews?test=True


{'created_at': None,
 'description': 'The greatest systematic review ever created.',
 'id': None,
 'name': "Burton's Great Review",
 'num_citation_screening_reviewers': None,
 'num_fulltext_screening_reviewers': None,
 'owner_user_id': 1,
 'status': None}

### get review's info by id

input: review id

output: review info, or a subset

In [24]:
call_endpoint('GET', 'reviews/{id}'.format(id=1), auth=auth)

GET => http://localhost:5000/reviews/1


{'created_at': '2016-09-29T23:52:29.266246+00:00',
 'description': 'International policy has sought to emphasize and strengthen the link between the conservation of natural ecosystems and human development. Furthermore, international conservation organizations have broadened their objectives beyond nature-based goals to recognize the contribution of conservation interventions in sustaining ecosystem services upon which human populations are dependent. While many indices have been developed to measure various human well-being domains, the strength of evidence to support the effects, both positive and negative, of conservation interventions on human well-being, is still unclear.\n\nThis protocol describes the methodology for examining the research question: What are the impacts of nature conservation interventions on different domains of human well-being in developing countries? Using systematic mapping, this study will scope and identify studies that measure the impacts of nature conser

### get info for all reviews visible to authenticated user

input: nothing

output: list of reviews' info, or subsets thereof

In [25]:
call_endpoint('GET', 'reviews', auth=auth,
              params={'fields': 'name,status'})

GET => http://localhost:5000/reviews?fields=name%2Cstatus


[{'id': 1, 'name': 'Conservation International Demo', 'status': 'active'}]

### change a review's info / settings

input: review id and field: value pairs to update

output: updated review info

In [26]:
modified_review_info = {'name': 'CI Demo',
                        'num_citation_screening_reviewers': 1}

call_endpoint('PUT', 'reviews/{id}'.format(id=1), auth=auth,
              json=modified_review_info, params={'test': True})

PUT => http://localhost:5000/reviews/1?test=True


{'created_at': '2016-09-29T23:52:29.266246+00:00',
 'description': 'International policy has sought to emphasize and strengthen the link between the conservation of natural ecosystems and human development. Furthermore, international conservation organizations have broadened their objectives beyond nature-based goals to recognize the contribution of conservation interventions in sustaining ecosystem services upon which human populations are dependent. While many indices have been developed to measure various human well-being domains, the strength of evidence to support the effects, both positive and negative, of conservation interventions on human well-being, is still unclear.\n\nThis protocol describes the methodology for examining the research question: What are the impacts of nature conservation interventions on different domains of human well-being in developing countries? Using systematic mapping, this study will scope and identify studies that measure the impacts of nature conser

### delete a review

input: review id

output: nothing

In [27]:
call_endpoint('DELETE', 'reviews/{id}'.format(id=1), auth=auth,
              params={'test': True})

DELETE => http://localhost:5000/reviews/1?test=True


---

## REVIEW TEAMS

### get the team of users on a review

input: review id

output: list of users on the review; owner is specially noted by `is_owner` field

In [28]:
call_endpoint('GET', 'reviews/{id}/team'.format(id=1), auth=auth)

GET => http://localhost:5000/reviews/1/team


[{'created_at': '2016-09-29T23:52:27.781970+00:00',
  'email': 'burtdewilde@gmail.com',
  'id': 1,
  'is_owner': True,
  'name': 'Burton DeWilde'},
 {'created_at': '2016-09-29T23:52:28.076214+00:00',
  'email': 'rayshah@thinkdesign.com',
  'id': 2,
  'name': 'Ray Shah'},
 {'created_at': '2016-09-29T23:52:28.368038+00:00',
  'email': 'caugustin@rsmas.miami.edu',
  'id': 3,
  'name': 'Caitlin Augustin'},
 {'created_at': '2016-09-29T23:52:28.665651+00:00',
  'email': 'rcm2164@columbia.edu',
  'id': 4,
  'name': 'Bob Minnich'},
 {'created_at': '2016-09-29T23:52:28.965896+00:00',
  'email': 'samanzaroot@gmail.com',
  'id': 5,
  'name': 'Sam Anzaroot'}]

### add or remove a user as a collaborator on a review

input: review id, user id, action type

output: updated list of users on the review

In [29]:
call_endpoint('PUT', 'reviews/{id}/team'.format(id=1),
              json={'user_id': 2, 'action': 'add'},
              params={'test': True},
              auth=auth)

PUT => http://localhost:5000/reviews/1/team?test=True


[{'created_at': '2016-09-29T23:52:27.781970+00:00',
  'email': 'burtdewilde@gmail.com',
  'id': 1,
  'is_owner': True,
  'name': 'Burton DeWilde'},
 {'created_at': '2016-09-29T23:52:28.076214+00:00',
  'email': 'rayshah@thinkdesign.com',
  'id': 2,
  'name': 'Ray Shah'},
 {'created_at': '2016-09-29T23:52:28.368038+00:00',
  'email': 'caugustin@rsmas.miami.edu',
  'id': 3,
  'name': 'Caitlin Augustin'},
 {'created_at': '2016-09-29T23:52:28.665651+00:00',
  'email': 'rcm2164@columbia.edu',
  'id': 4,
  'name': 'Bob Minnich'},
 {'created_at': '2016-09-29T23:52:28.965896+00:00',
  'email': 'samanzaroot@gmail.com',
  'id': 5,
  'name': 'Sam Anzaroot'}]

In [30]:
call_endpoint('PUT', 'reviews/{id}/team'.format(id=1),
              json={'user_id': 2, 'action': 'remove'},
              params={'test': True},
              auth=auth)

PUT => http://localhost:5000/reviews/1/team?test=True


[{'created_at': '2016-09-29T23:52:27.781970+00:00',
  'email': 'burtdewilde@gmail.com',
  'id': 1,
  'is_owner': True,
  'name': 'Burton DeWilde'},
 {'created_at': '2016-09-29T23:52:28.076214+00:00',
  'email': 'rayshah@thinkdesign.com',
  'id': 2,
  'name': 'Ray Shah'},
 {'created_at': '2016-09-29T23:52:28.368038+00:00',
  'email': 'caugustin@rsmas.miami.edu',
  'id': 3,
  'name': 'Caitlin Augustin'},
 {'created_at': '2016-09-29T23:52:28.665651+00:00',
  'email': 'rcm2164@columbia.edu',
  'id': 4,
  'name': 'Bob Minnich'},
 {'created_at': '2016-09-29T23:52:28.965896+00:00',
  'email': 'samanzaroot@gmail.com',
  'id': 5,
  'name': 'Sam Anzaroot'}]

### assign user as new review owner

input: review id, user id, action = 'make_owner'

output: updated list of users on the review

In [36]:
call_endpoint('PUT', 'reviews/{id}/team'.format(id=1),
              json={'user_id': 2, 'action': 'make_owner'},
              params={'test': True},
              auth=auth)

PUT => http://localhost:5000/reviews/1/team?test=True


[{'created_at': '2016-09-29T23:52:27.781970+00:00',
  'email': 'burtdewilde@gmail.com',
  'id': 1,
  'is_owner': True,
  'name': 'Burton DeWilde'},
 {'created_at': '2016-09-29T23:52:28.076214+00:00',
  'email': 'rayshah@thinkdesign.com',
  'id': 2,
  'name': 'Ray Shah'},
 {'created_at': '2016-09-29T23:52:28.368038+00:00',
  'email': 'caugustin@rsmas.miami.edu',
  'id': 3,
  'name': 'Caitlin Augustin'},
 {'created_at': '2016-09-29T23:52:28.665651+00:00',
  'email': 'rcm2164@columbia.edu',
  'id': 4,
  'name': 'Bob Minnich'},
 {'created_at': '2016-09-29T23:52:28.965896+00:00',
  'email': 'samanzaroot@gmail.com',
  'id': 5,
  'name': 'Sam Anzaroot'}]

---

## REVIEW PLANS

### get a review plan

input: review id, optional fields to return

output: review plan

In [32]:
call_endpoint('GET', 'reviews/{id}/plan'.format(id=1), auth=auth)

GET => http://localhost:5000/reviews/1/plan


{'data_extraction_form': {},
 'id': 1,
 'keyterms': [{'group': 'outcome',
   'synonyms': ['well-being', 'well being'],
   'term': 'wellbeing'},
  {'group': 'outcome',
   'synonyms': ['ecosystem services'],
   'term': 'ecosystem service'},
  {'group': 'outcome', 'synonyms': [], 'term': 'nutrition'},
  {'group': 'outcome', 'synonyms': ['skills'], 'term': 'skill'},
  {'group': 'outcome', 'synonyms': ['empowering'], 'term': 'empower'},
  {'group': 'outcome', 'synonyms': ['livelihood'], 'term': 'clean water'},
  {'group': 'outcome', 'synonyms': ['food security'], 'term': 'livelihoods'},
  {'group': 'outcome', 'synonyms': ['vulnerability'], 'term': 'resilience'},
  {'group': 'outcome', 'synonyms': ['social capital'], 'term': 'capital'},
  {'group': 'outcome', 'synonyms': ['attitudes'], 'term': 'attitude'},
  {'group': 'outcome', 'synonyms': ['perceptions'], 'term': 'perception'},
  {'group': 'outcome', 'synonyms': ['human health'], 'term': 'health'},
  {'group': 'outcome', 'synonyms': ['know

### add or modify parts of a review plan

input: review id, field: value pairs to modify

output: modified review plan

In [40]:
review_plan = {
    'outcome': 'foo bar bat baz',
    'pico': {'population': 'lots of people',
             'intervention': 'we did a thing',
             'comparison': 'we did not do a thing',
             'outcome': 'things got better'}
    }

call_endpoint('PUT', 'reviews/{id}/plan'.format(id=1),
              json=review_plan,
              params={'test': True},
              auth=auth)

PUT => http://localhost:5000/reviews/1/plan?test=True


{'data_extraction_form': {},
 'id': 1,
 'keyterms': [{'group': 'outcome',
   'synonyms': ['well-being', 'well being'],
   'term': 'wellbeing'},
  {'group': 'outcome',
   'synonyms': ['ecosystem services'],
   'term': 'ecosystem service'},
  {'group': 'outcome', 'synonyms': [], 'term': 'nutrition'},
  {'group': 'outcome', 'synonyms': ['skills'], 'term': 'skill'},
  {'group': 'outcome', 'synonyms': ['empowering'], 'term': 'empower'},
  {'group': 'outcome', 'synonyms': ['livelihood'], 'term': 'clean water'},
  {'group': 'outcome', 'synonyms': ['food security'], 'term': 'livelihoods'},
  {'group': 'outcome', 'synonyms': ['vulnerability'], 'term': 'resilience'},
  {'group': 'outcome', 'synonyms': ['social capital'], 'term': 'capital'},
  {'group': 'outcome', 'synonyms': ['attitudes'], 'term': 'attitude'},
  {'group': 'outcome', 'synonyms': ['perceptions'], 'term': 'perception'},
  {'group': 'outcome', 'synonyms': ['human health'], 'term': 'health'},
  {'group': 'outcome', 'synonyms': ['know

### delete an entire review plan

although why you would ever do this, I don't know... in fact we probably don't want to enable a "clear" option on this screen. nevertheless!

input: review id

output: nothing

In [41]:
call_endpoint('DELETE', 'reviews/{id}/plan'.format(id=1), auth=auth,
              params={'test': True})

DELETE => http://localhost:5000/reviews/1/plan?test=True


---

## Populate the Database

### Register new users via `POST` to `/users`

In [85]:
test = False
users_to_post = [
    {'name': 'Burton DeWilde', 'email': 'burtdewilde@gmail.com', 'password': 'password', 'test': test},
    {'name': 'Ray Shah', 'email': 'rayshah@thinkdesign.com', 'password': 'password', 'test': test},
    {'name': 'Caitlin Augustin', 'email': 'caugustin@rsmas.miami.edu', 'password': 'password', 'test': test},
    {'name': 'Bob Minnich', 'email': 'rcm2164@columbia.edu', 'password': 'password', 'test': test},
    {'name': 'Sam Anzaroot', 'email': 'samanzaroot@gmail.com', 'password': 'password', 'test': test}]

In [86]:
for user_to_post in users_to_post:
    user = call_endpoint('users', 'post', json=user_to_post)
    pprint(user)

http://localhost:5000/users
{'created_at': '2016-09-25T02:55:08.966457+00:00',
 'email': 'burtdewilde@gmail.com',
 'id': 1,
 'name': 'Burton DeWilde'}
http://localhost:5000/users
{'created_at': '2016-09-25T02:55:09.247314+00:00',
 'email': 'rayshah@thinkdesign.com',
 'id': 2,
 'name': 'Ray Shah'}
http://localhost:5000/users
{'created_at': '2016-09-25T02:55:09.533289+00:00',
 'email': 'caugustin@rsmas.miami.edu',
 'id': 3,
 'name': 'Caitlin Augustin'}
http://localhost:5000/users
{'created_at': '2016-09-25T02:55:09.813172+00:00',
 'email': 'rcm2164@columbia.edu',
 'id': 4,
 'name': 'Bob Minnich'}
http://localhost:5000/users
{'created_at': '2016-09-25T02:55:10.110419+00:00',
 'email': 'samanzaroot@gmail.com',
 'id': 5,
 'name': 'Sam Anzaroot'}


### Authenticate the current user against an existing user

In [87]:
auth = get_auth_token('burtdewilde@gmail.com', 'password')

http://localhost:5000/authtoken


### Create new reviews via `POST` to `/reviews`

In [88]:
test = False
reviews_to_post = [
    {'name': 'Conservation International Demo',
     'description': 'International policy has sought to emphasize and strengthen the link between the conservation of natural ecosystems and human development. Furthermore, international conservation organizations have broadened their objectives beyond nature-based goals to recognize the contribution of conservation interventions in sustaining ecosystem services upon which human populations are dependent. While many indices have been developed to measure various human well-being domains, the strength of evidence to support the effects, both positive and negative, of conservation interventions on human well-being, is still unclear.\n\nThis protocol describes the methodology for examining the research question: What are the impacts of nature conservation interventions on different domains of human well-being in developing countries? Using systematic mapping, this study will scope and identify studies that measure the impacts of nature conservation interventions on human well-being at local to regional scales. The primary objective of this study is to synthesize the state and distribution of the existing evidence base linking conservation and human well-being. In addition, a theory of change approach will be used to identify and characterize the causal linkages between conservation and human well-being, with attention on those studies that examine the role of ecosystem services. Key trends among the resulting studies will be synthesized and the range of studies organized and presented in a graphical matrix illustrating the relationships between types of interventions and types of outcomes. Results of the study are intended to help conservation and development practitioners and the academic community to improve research studies and conservation practices in developing countries in order to achieve both conservation and human well-being outcomes.',
     'test': test},
    {'name': "Burton's Great Systematic Review",
     'description': 'I do great systematic reviews. Really, really great.',
     'test': test}]

In [89]:
for review_to_post in reviews_to_post:
    review = call_endpoint('reviews', 'post', json=review_to_post, auth=auth)
    pprint(review)

http://localhost:5000/reviews
{'created_at': '2016-09-25T02:55:10.416993+00:00',
 'description': 'International policy has sought to emphasize and strengthen '
                'the link between the conservation of natural ecosystems and '
                'human development. Furthermore, international conservation '
                'organizations have broadened their objectives beyond '
                'nature-based goals to recognize the contribution of '
                'conservation interventions in sustaining ecosystem services '
                'upon which human populations are dependent. While many '
                'indices have been developed to measure various human '
                'well-being domains, the strength of evidence to support the '
                'effects, both positive and negative, of conservation '
                'interventions on human well-being, is still unclear.\n'
                '\n'
                'This protocol describes the methodology for examining

### Modify reviews via `PUT` to `/reviews/<id>`

Here, we require two reviewers per citation.

In [90]:
call_endpoint('reviews/1', 'put',
              json={'num_citation_screening_reviewers': 2, 'test': False},
              auth=auth)

http://localhost:5000/reviews/1


{'created_at': '2016-09-25T02:55:10.416993+00:00',
 'description': 'International policy has sought to emphasize and strengthen the link between the conservation of natural ecosystems and human development. Furthermore, international conservation organizations have broadened their objectives beyond nature-based goals to recognize the contribution of conservation interventions in sustaining ecosystem services upon which human populations are dependent. While many indices have been developed to measure various human well-being domains, the strength of evidence to support the effects, both positive and negative, of conservation interventions on human well-being, is still unclear.\n\nThis protocol describes the methodology for examining the research question: What are the impacts of nature conservation interventions on different domains of human well-being in developing countries? Using systematic mapping, this study will scope and identify studies that measure the impacts of nature conser

### Add review plans via `POST` to `/reviews/<id>/plan`

In [91]:
test = False
reviewplans_to_post = [
    {'test': test,
     'review_id': 1,
     'objective': 'To assess and characterize the current state and distribution of the existing evidence base around the causal linkages between both positive and negative effects of nature conservation and human well-being.',
     'research_questions': [
            "What are the impacts of nature conservation interventions on different domains of human well-being in developing countries?",
            "What is the current state and distribution of evidence?",
            "What types of impacts from conservation interventions on human well-being are measured?",
            "What types of ecosystem services are explicitly associated with the impacts of conservation interventions on human well-being?",
            "What populations are affected by conservation and/ or focus of studies?",
            "How does the evidence base align with major priorities and investments of implementing agencies?",
            ],
     'pico': {
            "population": "Human populations, including individuals, households, communities or nation states in non-OECD countries",
            "intervention": "Adoption or implementation of nature conservation interventions",
            "comparator": "No use of nature conservation interventions either between sites or groups, or over time series (before/after)",
            "outcome": "Positive or negative effects on the multi-dimensional well-being status of human populations",
            },
     'keyterms': [
            {"term": "wellbeing", "group": "outcome", "synonyms": ["well-being", "well being"]},
            {"term": "ecosystem service", "group": "outcome", "synonyms": ["ecosystem services"]},
            {"term": "nutrition", "group": "outcome"},
            {"term": "skill", "group": "outcome", "synonyms": ["skills"]},
            {"term": "empower", "group": "outcome", "synonyms": ["empowering"]},
            {"term": "clean water", "group": "outcome", "synonyms": ["livelihood"]},
            {"term": "livelihoods", "group": "outcome", "synonyms": ["food security"]},
            {"term": "resilience", "group": "outcome", "synonyms": ["vulnerability"]},
            {"term": "capital", "group": "outcome", "synonyms": ["social capital"]},
            {"term": "attitude", "group": "outcome", "synonyms": ["attitudes"]},
            {"term": "perception", "group": "outcome", "synonyms": ["perceptions"]},
            {"term": "health", "group": "outcome", "synonyms": ["human health"]},
            {"term": "human capital", "group": "outcome", "synonyms": ["knowledge"]},
            {"term": "traditional knowledge", "group": "outcome"},
            {"term": "marine", "group": "intervention qualifiers", "synonyms": ["freshwater"]},
            {"term": "coastal", "group": "intervention qualifiers"},
            {"term": "forest", "group": "intervention qualifiers", "synonyms": ["forests", "forestry"]},
            {"term": "ecosystem", "group": "intervention qualifiers", "synonyms": ["ecosystems"]},
            {"term": "species", "group": "intervention qualifiers"},
            {"term": "habitat", "group": "intervention qualifiers", "synonyms": ["habitats"]},
            {"term": "biodiversity", "group": "intervention qualifiers"},
            {"term": "sustainable", "group": "intervention qualifiers", "synonyms": ["sustainability"]},
            {"term": "ecology", "group": "intervention qualifiers", "synonyms": ["ecological"]},
            {"term": "integrated", "group": "intervention qualifiers"},
            {"term": "landscape", "group": "intervention qualifiers"},
            {"term": "seascape", "group": "intervention qualifiers"},
            {"term": "coral reef", "group": "intervention qualifiers", "synonyms": ["coral reefs"]},
            {"term": "natural resources", "group": "intervention qualifiers", "synonyms": ["natural resource"]},
            {"term": "human", "group": "outcome qualifiers", "synonyms": ["humans", "humanity"]},
            {"term": "people", "group": "outcome qualifiers"},
            {"term": "person", "group": "outcome qualifiers", "synonyms": ["persons"]},
            {"term": "community", "group": "outcome qualifiers", "synonyms": ["communities"]},
            {"term": "household", "group": "outcome qualifiers", "synonyms": ["households"]},
            {"term": "fishermen", "group": "outcome qualifiers", "synonyms": ["fisherman"]},
            {"term": "collaborative", "group": "outcome qualifiers"},
            {"term": "conservation", "group": "intervention"},
            {"term": "conserve", "group": "intervention"},
            {"term": "conservancy", "group": "intervention"},
            {"term": "protect", "group": "intervention", "synonyms": ["protects", "protection"]},
            {"term": "management", "group": "intervention"},
            {"term": "awareness", "group": "intervention"},
            {"term": "law", "group": "intervention", "synonyms": ["laws"]},
            {"term": "policy", "group": "intervention", "synonyms": ["policy-making"]},
            {"term": "reserve", "group": "intervention"},
            {"term": "govern", "group": "intervention", "synonyms": ["governs", "government"]},
            {"term": "capacity-build", "group": "intervention", "synonyms": ["capacity-building", "capacity building"]},
            {"term": "train", "group": "intervention", "synonyms": ["tarins", "training"]},
            {"term": "PES", "group": "intervention"},
            {"term": "ecotourism", "group": "intervention", "synonyms": ["eco-tourism"]},
            {"term": "sustainable use", "group": "intervention"}
            ],
     'selection_criteria': [
            {"label": "location",
             "description": "individuals, households, or communities must be in non-OECD countries"},
            {"label": "undefined pop.",
             "description": "studies must include discrete populations and not undefined groups or populations"},
            {"label": "intervention type",
             "description": "study must document or measure people's specific and discrete external interventions, not daily use"},
            {"label": "in-situ",
             "description": "study must focus on establishment, adoption, or implementation of regulation, protection, or management of natural ecosystems through in-situ activities"},
            {"label": "outcome",
             "explanation": "studies must measure or describe human well-being outcomes, and can't only focus on biophysical outcomes of conservation"}
            ],
     }
]

In [92]:
for reviewplan_to_post in reviewplans_to_post:
    review_id = reviewplan_to_post.pop('review_id')
    reviewplan = call_endpoint(
        'reviews/{}/plan'.format(review_id), 'post',
        json=reviewplan_to_post, auth=auth)
    pprint(reviewplan)

http://localhost:5000/reviews/1/plan
{'data_extraction_form': {},
 'id': 1,
 'keyterms': [{'group': 'outcome',
               'synonyms': ['well-being', 'well being'],
               'term': 'wellbeing'},
              {'group': 'outcome',
               'synonyms': ['ecosystem services'],
               'term': 'ecosystem service'},
              {'group': 'outcome', 'synonyms': [], 'term': 'nutrition'},
              {'group': 'outcome', 'synonyms': ['skills'], 'term': 'skill'},
              {'group': 'outcome',
               'synonyms': ['empowering'],
               'term': 'empower'},
              {'group': 'outcome',
               'synonyms': ['livelihood'],
               'term': 'clean water'},
              {'group': 'outcome',
               'synonyms': ['food security'],
               'term': 'livelihoods'},
              {'group': 'outcome',
               'synonyms': ['vulnerability'],
               'term': 'resilience'},
              {'group': 'outcome',
          

### Add (or remove) collaborators to review via `PUT` to `/reviews/<id>/team`

In [93]:
call_endpoint('reviews/1/team', 'put',
              json={'user_id': 2, 'action': 'add', 'test': False},
              auth=auth)

http://localhost:5000/reviews/1/team


[{'created_at': '2016-09-25T02:55:08.966457+00:00',
  'email': 'burtdewilde@gmail.com',
  'id': 1,
  'is_owner': True,
  'name': 'Burton DeWilde'},
 {'created_at': '2016-09-25T02:55:09.247314+00:00',
  'email': 'rayshah@thinkdesign.com',
  'id': 2,
  'name': 'Ray Shah'}]

### Upload collections of citations via `POST` (files in request) to `/citations`

Let's split it into two files to simulate "batches". When it's implemented (soon...), this should kick off separate, asynchronous deduplication threads.

In [94]:
test = False
filepaths = ['../data/raw/citation_files/ci-full-collection-group1.ris',
             '../data/raw/citation_files/ci-full-collection-group2.ris']
# filepaths = ['../data/raw/citation_files/ci-full-collection.ris']
# filepaths = ['../data/raw/citation_files/ci-full-collection.bib']

for filepath in filepaths:
    filename = os.path.split(filepath)[-1]
    call_endpoint('citations', 'post',
                  data={'review_id': 1, 'test': test},
                  files={'uploaded_file': (filename, io.open(filepath, mode='rb'))},
                  auth=auth)

http://localhost:5000/citations
http://localhost:5000/citations


### Screen citations via `POST` to `/citations/screenings`

Let's simulate a two-reviewer setup, and randomly change screening decisions as well as whether to screen or not.

The screenings data was processed from CI's submitted data, is not complete, and given all the database changes, maybe totally wrong. Will update at some point. Meanwhile, it's fine for practice data.

**Note:** This process takes a few minutes. In typical usage, a user submits one screening at a time, not 28,000.

In [245]:
%%time
test = False

selection_criteria = call_endpoint(
    'reviews/1/plan', 'get',
    params={'fields': 'selection_criteria'},
    auth=('burtdewilde@gmail.com', 'password')).get('selection_criteria')
all_exclude_reasons = [criterion['label'] for criterion in selection_criteria]
    
for auth in [('rayshah@thinkdesign.com', 'password'), ('burtdewilde@gmail.com', 'password')]:
    auth_token = (call_endpoint('authtoken', auth=auth)['token'], '')

    filepath = '../data/processed/citation_selection.json'
    with io.open(filepath, mode='rt') as f:
        screenings = json.load(f)
    # let's only screen a random 75% of the total per reviewer
    screenings = [screening for screening in screenings
                  if random.random() > 0.25
                  and screening['citation_id'] < 28708]  # <= HACK
    
    for screening in screenings:
        screening['status'] = 'included' if screening.pop('included') is True else 'excluded'
        # let's switch the decision for a random 5% of screenings
        if random.random() > 0.95:
            screening['status'] = 'included' if screening['status'] == 'excluded' else 'excluded'
        # if excluded, let's get a random sample of exclude reasons
        if screening['status'] == 'excluded':
            screening['exclude_reasons'] = random.sample(
                all_exclude_reasons, random.randint(1, 2))
            
    call_endpoint('citations/screenings', 'post',
                  json=screenings, params={'review_id': 1, 'test': test},
                  auth=auth)

http://localhost:5000/reviews/1/plan?fields=selection_criteria
http://localhost:5000/authtoken
http://localhost:5000/citations/screenings?review_id=1&test=False
http://localhost:5000/authtoken
http://localhost:5000/citations/screenings?review_id=1&test=False
CPU times: user 498 ms, sys: 17.4 ms, total: 515 ms
Wall time: 19.3 s


In [235]:
# call_endpoint('citations/1/screenings', 'post', json={'status': 'included'},
#               auth=('burtdewilde@gmail.com', 'password'))

# call_endpoint('citations/1/screenings', 'post', json={'status': 'included'},
#               auth=('rayshah@thinkdesign.com', 'password'))

# call_endpoint('citations/1/screenings', 'delete',
#               auth=('rayshah@thinkdesign.com', 'password'))

# call_endpoint('citations/1/screenings', 'delete',
#               auth=('burtdewilde@gmail.com', 'password'))

http://localhost:5000/citations/1/screenings
http://localhost:5000/citations/1/screenings


In [217]:
# # NOTE: This is the "correct" way to add citation screenings via the REST API.
# # The hack above is done purely for speed while in development.

# test = False
# filepath = '../data/processed/citation_selection.json'
# with io.open(filepath, mode='rt') as f:
#     screenings = json.load(f)

# selection_criteria = call_endpoint(
#     'reviews/1/plan', 'get',
#     params={'fields': 'selection_criteria'},
#     auth=('burtdewilde@gmail.com', 'password')).get('selection_criteria')
# all_exclude_reasons = [criterion['label'] for criterion in selection_criteria]
    
# for auth in [('rayshah@thinkdesign.com', 'password'), ('burtdewilde@gmail.com', 'password')]:
#     auth_token = (call_endpoint('authtoken', auth=auth)['token'], '')

#     for screening in screenings[:200]:
#         try:
#             # let's only screen a random 75% of the total per reviewer
#             if random.random() > 0.75:
#                 continue
#             citation_id = screening['citation_id']
#             status = 'included' if screening['included'] is True else 'excluded'
#             # let's switch the decision for a random 5% of screenings
#             if random.random() > 0.95:
#                 status = 'included' if status == 'excluded' else 'excluded'
#             screening_to_post = {'status': status}
#             if status == 'excluded':
#                 screening_to_post['exclude_reasons'] = random.sample(all_exclude_reasons, random.randint(1, 2))
#             screening_to_post['test'] = test
#             call_endpoint('citations/{}/screenings'.format(citation_id), 'post',
#                           json=screening_to_post,
#                           auth=auth, verbose=False)
#         except Exception as e:
#             continue

### Get fulltexts

In [302]:
auth = get_auth_token('burtdewilde@gmail.com', 'password')

http://localhost:5000/authtoken


In [296]:
fulltexts = call_endpoint('fulltexts', 'get',
                          params={'review_id': 1, 'fields': 'citation.title,status,filename'},
                          auth=auth)
fulltexts

http://localhost:5000/fulltexts?review_id=1&fields=citation.title%2Cstatus%2Cfilename


[{'citation': {'title': 'Rural development: from vision to action. A sector strategy'},
  'filename': None,
  'id': 465,
  'status': 'not_screened'},
 {'citation': {'title': 'The winter habitat selection of red panda (Ailurus fulgens) in the meigu dafengding national nature reserve, china'},
  'filename': None,
  'id': 464,
  'status': 'not_screened'},
 {'citation': {'title': 'Separating adaptive maintenance (Resilience) and transformative capacity of social-ecological systems'},
  'filename': None,
  'id': 463,
  'status': 'not_screened'},
 {'citation': {'title': 'Negotiating TVET for sustainable livelihoods'},
  'filename': None,
  'id': 462,
  'status': 'not_screened'},
 {'citation': {'title': 'The farmer, the worker and the MP: The digital divide and territorial paradoxes in Switzerland'},
  'filename': None,
  'id': 461,
  'status': 'not_screened'},
 {'citation': {'title': 'A recent inventory of the fishes of the north-western and central western coast of Lake Tanganyika (Democrat

### Upload fulltext PDF or TXT via `POST` to `/fulltexts/<id>/upload`

In [308]:
test = False

filepath = '/Users/burtondewilde/Desktop/Abstractive_Summarization_with_Attentive_RNN_-_NAACL_2016.pdf'
filename = 'Abstractive_Summarization_with_Attentive_RNN_-_NAACL_2016.pdf'

for fulltext in fulltexts[:1]:
    call_endpoint('fulltexts/{}/upload'.format(fulltext['id']), 'post',
                  data={'test': test},
                  files={'uploaded_file': (filename, io.open(filepath, mode='rb'))},
                  auth=auth, verbose=False)
    
    uploaded_file = call_endpoint('fulltexts/{}/upload'.format(fulltext['id']),
                                  'get', auth=auth, return_as_json=False)
    
# for fulltext in fulltexts:
#     call_endpoint('fulltexts/{}/upload'.format(fulltext['id']), 'delete',
#                   data={'test': test},
#                   auth=auth, verbose=False)

http://localhost:5000/fulltexts/465/upload


---

## Get Records in the Database

### Get a collection of citations via `GET` to `/citations`

In [273]:
call_endpoint('citations', 'get',
              params={'review_id': 1,
                      'fields': 'title,status',
                      'status': 'awaiting_coscreener',
                      'tag': None,
                      #'tsquery': 'fisheries',
                      'order_dir': 'ASC', 'per_page': 10},
              auth=auth)

http://localhost:5000/citations?review_id=1&status=awaiting_coscreener&fields=title%2Cstatus&per_page=10&order_dir=ASC


[{'id': 4,
  'status': 'screened_once',
  'title': 'Crisis and conservation at the end of the world: Sheep ranching in Argentine Patagonia'},
 {'id': 5,
  'status': 'screened_once',
  'title': 'Susceptibility of different bacterial species isolated from food animals to copper sulphate, zinc chloride and antimicrobial substances used for disinfection'},
 {'id': 12,
  'status': 'screened_once',
  'title': 'A conceptual framework of adoption of an agricultural innovation'},
 {'id': 14,
  'status': 'screened_once',
  'title': 'Managing the Brisbane River and Moreton Bay: An integrated research/management program to reduce impacts on an Australian estuary'},
 {'id': 22,
  'status': 'screened_once',
  'title': 'Engaging students in active learning: Use of a blog and audience response system'},
 {'id': 28,
  'status': 'screened_once',
  'title': 'Botanical ethnoveterinary therapies in three districts of the Lesser Himalayas of Pakistan'},
 {'id': 29,
  'status': 'screened_once',
  'title': 'E

### Get a collection of fulltexts via `GET` to `fulltexts`

In [24]:
call_endpoint('fulltexts', 'get',
              params={'review_id': 1,
                      'fields': None,
                      'status': None,
                      'tag': None,
                      'tsquery': None,
                      'order_dir': 'ASC', 'per_page': 10},
              auth=auth)

http://localhost:5000/fulltexts?per_page=10&order_dir=ASC&review_id=1


[{'citation_id': 18,
  'content': None,
  'created_at': '2016-09-24T01:50:00.205776+00:00',
  'extracted_info': None,
  'filename': '18.pdf',
  'id': 1,
  'status': 'not_screened'}]

In [48]:
auth = get_auth_token('burtdewilde@gmail.com', 'password')

http://localhost:5000/authtoken


In [9]:
call_endpoint('fulltexts', 'get',
              params={'review_id': 1,
                      'fields': None,
                      'status': None,
                      'tag': None,
                      'tsquery': None,
                      'order_dir': 'ASC', 'per_page': 10},
              auth=auth)

http://localhost:5000/fulltexts?review_id=1&per_page=10&order_dir=ASC


[]

### Get a single citation via `GET` to the `citations/<id>` endpoint

In [49]:
call_endpoint('citations/100', 'get',
              params={'fields': 'fulltext'}, # 'title,abstract,authors,keywords,pub_year,screenings'},
              auth=auth)

http://localhost:5000/citations/100?fields=fulltext


{'fulltext': None}

In [129]:
result = call_endpoint('citations/screenings', 'get',
                       params={'user_id': 1, 'review_id': 1, 'status_counts': True},
                       auth=auth)
print(len(result))
result

http://localhost:5000/citations/screenings?user_id=1&review_id=1&status_counts=True
2


{'excluded': 27627, 'included': 715}

In [137]:
result = call_endpoint('citations/screenings/1', 'get',
                       params={'fields': 'user_id,status'},
                       auth=auth)
result

http://localhost:5000/citations/screenings/1?fields=user_id%2Cstatus


{'status': 'excluded', 'user_id': 1}

In [138]:
result = call_endpoint('citations/screenings/1', 'delete',
                       params={'test': True},
                       auth=auth)
result

http://localhost:5000/citations/screenings/1?test=True


In [246]:
pprint(call_endpoint('reviews/1/progress', 'get',
              params={'step': 'all', 'user_view': False},
              auth=('burtdewilde@gmail.com', 'password')))

http://localhost:5000/reviews/1/progress?step=all&user_view=False
{'citations': {'conflict': 1484,
               'excluded': 14159,
               'included': 417,
               'not_screened': 2128,
               'screened_once': 10520},
 'fulltexts': {'not_screened': 417},
 'planning': {'keyterms': True,
              'objective': True,
              'pico': True,
              'selection_criteria': True}}


In [247]:
pprint(call_endpoint('reviews/1/progress', 'get',
              params={'step': 'all', 'user_view': True},
              auth=('burtdewilde@gmail.com', 'password')))

http://localhost:5000/reviews/1/progress?step=all&user_view=True
{'citations': {'awaiting_coscreener': 5217,
               'conflict': 1484,
               'excluded': 14159,
               'included': 417,
               'pending': 7431},
 'fulltexts': {'pending': 417},
 'planning': {'keyterms': True,
              'objective': True,
              'pico': True,
              'selection_criteria': True}}


In [205]:
# call_endpoint('reviews/1/team', 'put',
#               json={'user_id': 2, 'action': 'remove', 'test': False},
#               auth=auth_token)

# call_endpoint('reviews/1/team', 'put',
#               json={'user_id': 2, 'action': 'make_owner', 'test': False},
#               auth=auth_token)

# call_endpoint('reviews/1/team', 'get',
#               params={'fields': 'name'},
#               auth=auth_token)

In [6]:
from colandr.lib import utils
import logging

logging.basicConfig()
logging.getLogger('sqlalchemy.engine').setLevel(logging.CRITICAL)

list(utils.execute_raw_sql_query('SELECT COUNT(1) FROM citations'))

2016-09-15 16:07:32,618 INFO sqlalchemy.engine.base.Engine select version()


INFO:sqlalchemy.engine.base.Engine:select version()


2016-09-15 16:07:32,619 INFO sqlalchemy.engine.base.Engine {}


INFO:sqlalchemy.engine.base.Engine:{}


2016-09-15 16:07:32,620 INFO sqlalchemy.engine.base.Engine select current_schema()


...

2016-09-15 16:07:32,628 INFO sqlalchemy.engine.base.Engine SELECT COUNT(1) FROM citations


INFO:sqlalchemy.engine.base.Engine:SELECT COUNT(1) FROM citations


2016-09-15 16:07:32,629 INFO sqlalchemy.engine.base.Engine {}


INFO:sqlalchemy.engine.base.Engine:{}


[(28708,)]

In [93]:
call_endpoint('citations/1/screenings', 'post',
              data={'status': 'included'},
              auth=('burtdewilde@gmail.com', 'password'))

http://localhost:5000/citations/1/screenings


{'citation_id': 1,
 'created_at': '2016-09-24T19:47:57.600966+00:00',
 'exclude_reasons': None,
 'id': 1,
 'review_id': 1,
 'status': 'included',
 'user_id': 1}

In [95]:
call_endpoint('citations/1/screenings', 'post',
              data={'status': 'included'},
              auth=('rayshah@thinkdesign.com', 'password'))

http://localhost:5000/citations/1/screenings


{'citation_id': 1,
 'created_at': '2016-09-24T19:48:42.372801+00:00',
 'exclude_reasons': None,
 'id': 2,
 'review_id': 1,
 'status': 'included',
 'user_id': 2}

In [237]:
%%time
import itertools
from operator import itemgetter

from flask import current_app, g
from sqlalchemy import asc, desc

from colandr import create_app, db
from colandr.models import Citation, CitationScreening, Review, User
from colandr.api.schemas import CitationSchema

logging.getLogger('sqlalchemy.engine').setLevel(logging.WARNING)
app = create_app('default')

citation_ids = list(range(1, 25, 5))

with app.app_context():
    
    with db.engine.connect() as connection:
        query = """
            SELECT citation_id, ARRAY_AGG(status)
            FROM citation_screenings
            WHERE citation_id IN ({citation_ids})
            GROUP BY citation_id
            """.format(citation_ids=','.join(str(cid) for cid in citation_ids))
        print(list(connection.execute(query)))

    
#     add_fulltexts_query = """
#         SELECT citations.id
#         FROM citations
#         LEFT JOIN fulltexts ON citations.id = fulltexts.citation_id
#         WHERE
#             citations.id IN ({citation_ids})
#             AND citations.status = 'included'
#             AND fulltexts.id IS NULL
#         ORDER BY citations.id
#         """.format(citation_ids=','.join(str(cid) for cid in range(1000)))
#     print([row[0] for row in db.engine.execute(add_fulltexts_query)])


#     results = db.session.query(CitationScreening)\
#         .filter(CitationScreening.citation_id.in_(citation_ids))
        
#     for key, grp in itertools.groupby(results, lambda x: x.citation_id):
#         print(key, list(grp))
        
#     print(results)
    
#     for citation_id in citation_ids:
#         citation = db.session.query(Citation).get(citation_id)
#     citations = db.session.query(Citation)\
#         .filter(Citation.id.in_(citation_ids))\
#         .with_entities(Citation.id, Citation.status)\
#         .order_by(Citation.id)\
#         .all()
#     results = db.session.query(Citation, CitationScreening)\
#         .filter(Citation.id == CitationScreening.citation_id)\
#         .filter(Citation.id.in_(citation_ids))\
#         .all()
#     print(results[0])
#     query = db.session.query(Citation)\
#         .filter(Citation.id.in_(citation_ids))
#     for result in query:
#         print(result)

2016-09-25 14:52:33,505 INFO sqlalchemy.engine.base.Engine select version()


INFO:sqlalchemy.engine.base.Engine:select version()


2016-09-25 14:52:33,506 INFO sqlalchemy.engine.base.Engine {}


INFO:sqlalchemy.engine.base.Engine:{}


2016-09-25 14:52:33,507 INFO sqlalchemy.engine.base.Engine select current_schema()


...

2016-09-25 14:52:33,516 INFO sqlalchemy.engine.base.Engine 
            SELECT citation_id, ARRAY_AGG(status)
            FROM citation_screenings
            WHERE citation_id IN (1,6,11,16,21)
            GROUP BY citation_id
            


INFO:sqlalchemy.engine.base.Engine:
            SELECT citation_id, ARRAY_AGG(status)
            FROM citation_screenings
            WHERE citation_id IN (1,6,11,16,21)
            GROUP BY citation_id
            


2016-09-25 14:52:33,517 INFO sqlalchemy.engine.base.Engine {}


INFO:sqlalchemy.engine.base.Engine:{}


[(6, ['included']), (11, ['excluded']), (21, ['excluded', 'excluded']), (1, ['excluded']), (16, ['excluded'])]
CPU times: user 18.5 ms, sys: 3.09 ms, total: 21.6 ms
Wall time: 30.5 ms


In [143]:
from flask import current_app, g
from sqlalchemy import asc, desc

from colandr import create_app, db
from colandr.models import Citation, Review, User
from colandr.api.schemas import CitationSchema

logging.getLogger('sqlalchemy.engine').setLevel(logging.WARNING)
app = create_app('default')

review_id = 1
fields = None  # ['title']
status = 'not_screened'
tag = None
order_by = 'recency'
order_dir = 'DESC'
page = 0
per_page = 25

with app.app_context():
    
    user = db.session.query(User).get(1)
    print(user)
    print(any(review.users.filter_by(id=2).one_or_none()
              for review in user.reviews))
    
#     result = db.session.query(Citation).filter_by(id=100)
#     status = 'not_screened' OR
#     query = """
#         SELECT
#             (CASE
#                  WHEN (status != 'not_screened' AND screening @> '[{{"user_id": {user_id}}}]') THEN 'awaiting_coscreener'
#                  WHEN (status = 'not_screened' OR NOT screening @> '[{{"user_id": {user_id}}}]') THEN 'pending'
#              END) AS user_view_status,
#             COUNT(1)
#         FROM citations
#         GROUP BY 1
#         """.format(user_id=1)
#     results = [row for row in db.engine.execute(query)]
#         print(result)
#         db.text("SELECT id, screening FROM citations WHERE screening @> '[\{\"user_id\":{}\}]'".format(1))).all()
#     print(result)

2016-09-16 14:47:31,000 INFO sqlalchemy.engine.base.Engine select version()


INFO:sqlalchemy.engine.base.Engine:select version()


2016-09-16 14:47:31,001 INFO sqlalchemy.engine.base.Engine {}


INFO:sqlalchemy.engine.base.Engine:{}


2016-09-16 14:47:31,003 INFO sqlalchemy.engine.base.Engine select current_schema()


...

INFO:sqlalchemy.engine.base.Engine:SELECT users.id AS users_id, users.created_at AS users_created_at, users.name AS users_name, users.email AS users_email, users.password AS users_password 
FROM users, users_to_reviews 
WHERE %(param_1)s = users_to_reviews.review_id AND users.id = users_to_reviews.user_id AND users.id = %(id_1)s


2016-09-16 14:47:31,028 INFO sqlalchemy.engine.base.Engine {'id_1': 2, 'param_1': 2}


INFO:sqlalchemy.engine.base.Engine:{'id_1': 2, 'param_1': 2}


False
2016-09-16 14:47:31,029 INFO sqlalchemy.engine.base.Engine COMMIT


INFO:sqlalchemy.engine.base.Engine:COMMIT


Get one or more existing users:

In [113]:
call_endpoint('users', 'get', params={'email': 'burtdewilde@gmail.com'}, auth=auth_token)

http://localhost:5000/users?email=burtdewilde%40gmail.com


{'created_at': '2016-09-10T03:59:26.298284+00:00',
 'email': 'burtdewilde@gmail.com',
 'id': 1,
 'name': 'Burton DeWilde'}

Update an existing user:

In [114]:
call_endpoint('users/1', 'put',
              data={'email': 'burtondewilde@gmail.com', 'test': False},
              auth=auth_token)

http://localhost:5000/users/1


{'created_at': '2016-09-10T03:59:26.298284+00:00',
 'email': 'burtondewilde@gmail.com',
 'id': 1,
 'name': 'Burton DeWilde'}

Update an existing review:

In [118]:
call_endpoint('reviews/2', 'put',
              data={'name': "Burton's Greatest Systematic Review"}, auth=auth_token)

http://localhost:5000/reviews/2


{'created_at': '2016-09-10T03:59:51.318202+00:00',
 'description': 'I do great systematic reviews. Really, really great.',
 'id': 2,
 'name': "Burton's Greatest Systematic Review",
 'num_citation_screening_reviewers': 1,
 'num_fulltext_screening_reviewers': 1,
 'owner_user_id': 1,
 'status': 'active'}

Update an existing review plan:

In [70]:
call_endpoint('reviewplans/1', 'put',
              json={'objective': 'Eat food. Mostly plants. Not too much.'},
              auth=auth_token)

http://localhost:5000/reviewplans/1


{'data_extraction_form': {},
 'id': 1,
 'keyterms': [],
 'objective': 'Eat food. Mostly plants. Not too much.',
 'pico': {'intervention': 'Adoption or implementation of nature conservation interventions',
  'outcome': 'Positive or negative effects on the multi-dimensional well-being status of human populations',
  'population': 'Human populations, including individuals, households, communities or nation states in non-OECD countries'},
 'research_questions': [],
 'review_id': 1,
 'selection_criteria': []}

In [117]:
call_endpoint('reviews', 'get', params={'fields': 'name'}, auth=auth_token)

http://localhost:5000/reviews?fields=name


[{'name': 'Conservation International Demo'},
 {'name': "Burton's Great Systematic Review"}]

In [135]:
# call_endpoint('reviews', 'get', params={'fields': 'name,created_at'}, auth=auth_token)

In [134]:
# call_endpoint('reviews/2', 'delete', data={'test': True}, auth=auth_token)

In [36]:
call_endpoint('users', 'get', params={'email': 'burtdewilde@gmail.com'}, auth=auth_token)

http://localhost:5000/users?email=burtdewilde%40gmail.com


{'created_at': '2016-09-05T22:38:44.633619+00:00',
 'email': 'burtdewilde@gmail.com',
 'id': 1,
 'name': 'Burton DeWilde'}

In [77]:
# call_endpoint('users', 'get', params={'review_id': 1})  # TODO

In [10]:
call_endpoint('users/1', 'get', params={'fields': 'name'}, auth=auth)

INFO:requests.packages.urllib3.connectionpool:Resetting dropped connection: localhost


http://localhost:5000/users/1?fields=name


{'name': 'Burton DeWilde'}

In [11]:
call_endpoint('users/1', 'delete', params={'test': False})

INFO:requests.packages.urllib3.connectionpool:Resetting dropped connection: localhost


http://localhost:5000/users/1?test=False


HTTPError: 500 Server Error: INTERNAL SERVER ERROR for url: http://localhost:5000/users/1?test=False

---

## Reviews

#### LOGIN

In [5]:
call_endpoint('login', auth=('burtdewilde@gmail.com', 'test'))

http://localhost:5000/login


{'email': 'burtdewilde@gmail.com',
 'name': 'Burton DeWilde',
 'owned_review_ids': [1],
 'review_ids': [1],
 'user_id': 1}

In [6]:
call_endpoint('users/1', 'get')

http://localhost:5000/users/1


{'created_ts': '2016-07-07T17:56:07+00:00',
 'email': 'burtdewilde@gmail.com',
 'name': 'Burton DeWilde',
 'owned_review_ids': [1],
 'review_ids': [1],
 'user_id': 1}

In [7]:
call_endpoint('users/2', 'get')

http://localhost:5000/users/2


HTTPError: 500 Server Error: INTERNAL SERVER ERROR for url: http://localhost:5000/users/2

In [27]:
call_endpoint('users', 'post',
              data={'name': 'Ray Shah',
                    'email': 'rayshah@thinkdesign.com',
                    'password': 'test',
                    'review_ids': None,
                    'owned_review_ids': None,
                    'test': True})

http://localhost:5000/users


{'email': 'rayshah@thinkdesign.com',
 'name': 'Ray Shah',
 'owned_review_ids': None,
 'password': 'test',
 'review_ids': None}

In [10]:
call_endpoint('users/1', 'delete',
              data={'test': True})

http://localhost:5000/users/1


In [51]:
call_endpoint('reviews', 'post',
              data={'name': 'My Great Review',
                    'description': 'foo bar bat baz',
                    'test': True})

http://localhost:5000/reviews


{'description': 'foo bar bat baz',
 'name': 'My Great Review',
 'owner_user_id': 1,
 'settings': '{"num_citation_screening_reviewers": 2, "required_citation_screener_id": null, "num_fulltext_screening_reviewers": 2, "required_fulltext_screener_id": null}',
 'user_ids': [1]}

In [58]:
call_endpoint('reviews/1', 'delete',
              params={'test': True})

http://localhost:5000/reviews/1?test=True


In [60]:
call_endpoint('users/6', 'delete')

http://localhost:5000/users/6


6

In [12]:
call_endpoint('reviews/1')

http://localhost:5000/reviews/1


{'created_ts': 'Wed, 13 Jul 2016 01:40:27 GMT',
 'description': 'International policy has sought to emphasize and strengthen the link between the conservation of natural ecosystems and human development. Furthermore, international conservation organizations have broadened their objectives beyond nature-based goals to recognize the contribution of conservation interventions in sustaining ecosystem services upon which human populations are dependent. While many indices have been developed to measure various human well-being domains, the strength of evidence to support the effects, both positive and negative, of conservation interventions on human well-being, is still unclear.\\n\\nThis protocol describes the methodology for examining the research question: What are the impacts of nature conservation interventions on different domains of human well-being in developing countries? Using systematic mapping, this study will scope and identify studies that measure the impacts of nature conserv